In [ ]:
# Imports
import numpy as np
import xarray as xr

# Read a restart file into an xarray Dataset object
ds = xr.open_dataset("GEOSChem.SpeciesConc.20190701_0000z.nc4")

# Print the contents of the DataSet
print(ds)

In [ ]:
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point

# Ouvrir le fichier NetCDF
ds = xr.open_dataset("GEOSChem.SpeciesConc.20190701_0000z.nc4")

# Liste des variables d'intérêt
variables_of_interest = ["SpeciesConcVV_SO2", "SpeciesConcVV_SO4", "SpeciesConcVV_NO", 
                         "SpeciesConcVV_NO2", "SpeciesConcVV_NO3", "SpeciesConcVV_HNO2", 
                         "SpeciesConcVV_HNO3", "SpeciesConcVV_NH3"]

# Filtrer les variables présentes dans le fichier NetCDF
variables_presentes = [var for var in variables_of_interest if var in ds.variables]

# Récupération des coordonnées
lons = ds['lon'].values
lats = ds['lat'].values

# Création d'une GeoDataFrame pour chaque variable
for var in variables_presentes:
    # Extraire la variable sous forme de DataArray et moyenner sur les niveaux temporels et verticaux si nécessaire
    data = ds[var].mean(dim=['time', 'lev'])

    # Conversion des données en un DataFrame
    points = []
    values = []
    for i, lon in enumerate(lons):
        for j, lat in enumerate(lats):
            points.append(Point(lon, lat))
            values.append(data[j, i].values)  # Notez l'ordre (lat, lon)

    # Création de la GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': points, var: values}, crs="EPSG:4326")

    # Sauvegarde en fichier .shp
    gdf.to_file(f"{var}.shp")

print("Fichiers SHP créés avec succès.")


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Charger le fichier NetCDF
ds = xr.open_dataset("GEOSChem.SpeciesConc.20190701_0000z.nc4")

# Liste des substances d'intérêt
variables_of_interest = ["SpeciesConcVV_SO2", "SpeciesConcVV_SO4", "SpeciesConcVV_NO", 
                         "SpeciesConcVV_NO2", "SpeciesConcVV_NO3", "SpeciesConcVV_HNO2", 
                         "SpeciesConcVV_HNO3", "SpeciesConcVV_NH3"]

# Vérification des variables disponibles dans le fichier NetCDF
variables_presentes = [var for var in variables_of_interest if var in ds.variables]

# Coordonnées pour les longitudes et latitudes
lons = ds['lon'].values
lats = ds['lat'].values

# Création d'une carte pour chaque variable
for var in variables_presentes:
    # Extraire la moyenne de la concentration sur les dimensions temporelles et verticales
    data = ds[var].mean(dim=['time', 'lev'])

    # Création de la figure et du système de projection
    plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()  # Pour afficher la carte complète
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Tracer les données en tant que carte de chaleur
    plt.pcolormesh(lons, lats, data, transform=ccrs.PlateCarree(), cmap='viridis')
    plt.colorbar(label=f"{var} concentration")  # Barre de couleurs indiquant la concentration

    # Ajouter un titre et sauvegarder l'image
    plt.title(f"Concentration de {var} (moyenne)")
    plt.savefig(f"{var}_concentration_map.png")
    plt.close()

print("Images générées avec succès.")


/home/edgar/gcmanage/lib/python3.12/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/home/edgar/gcmanage/lib/python3.12/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Images générées avec succès.
